In [1]:
import pandas as pd
import numpy as np


In [2]:
df= pd.read_excel('Dataset.xlsx') 



In [3]:
df.head()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
0,1,disgust,নির্বাচনের আগের রাতে সুষ্টু শান্তিপূর্ণ ভাবে ভ...,2016,NaN,NaN,NaN
1,2,disgust,জনগণ মনের আনন্দে নিজেদের ভোট কাস্ট করে দেশদ্রো...,2016,NaN,NaN,NaN
2,3,angry,ভেজাল নির্বাচন ।,2016,NaN,NaN,NaN
3,4,happy,বিএনপি ও রাজাকার সমর্থকরা হেরে গেলে দেশের জন্য...,2016,NaN,NaN,NaN
4,5,angry,"সূক্ষ্ম কারচুপি বলে এতদিন একটা কথার কথা ছিল, এ...",2016,NaN,NaN,NaN


In [4]:
df.tail()

,SL,Emotion,Text,Year,Racial Biases,Hatred Percentage,Depression Percentage
35995,35996,surprise,ভারতের মতো একটা বিশাল দেশে এত অঙ্গরাজ্য যে সব ...,2021,NaN,NaN,NaN
35996,35997,surprise,"২৪, ৫০, এমনকি কারো কারো মতে ৬০ জন।",2021,NaN,NaN,NaN
35997,35998,surprise,এদের মধ্য থেকে ২৭ জনকে রাজা বিয়ে দিয়েছিল চন্দ্...,2021,NaN,NaN,NaN
35998,35999,surprise,"ওয়াকফের শাব্দিক অর্থ কোনো কিছু আটকে রাখা, উৎসর...",2021,NaN,NaN,NaN
35999,36000,surprise,তৃণমূলের সাবেক মন্ত্রী শুভেন্দু সম্প্রতি মমতা...,2021,NaN,NaN,NaN


In [5]:
df["Emotion"].value_counts()

disgust     6000
angry       6000
happy       6000
surprise    6000
fear        6000
sad         6000
Name: Emotion, dtype: int64

In [6]:
class_names = ['disgust','angry','happy','surprise','fear','sad']

In [8]:
encoding = {
    'disgust': 0, 
    'angry' :1,
    'happy':2,
    'surprise':3,
    'fear':4,
    'sad':5
}

In [7]:
reviews = df['Text'].values.tolist()
sentiments = df['Emotion'].tolist()

In [8]:
from sklearn.model_selection import train_test_split,KFold
X_train, X_test, y_train, y_test = train_test_split(reviews , sentiments, test_size=.2)

In [11]:
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [9]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-multilingual-cased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [10]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 67


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 43
	99percentile : 66


In [11]:
model = t.get_classifier()

In [12]:
learner1 = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
learner1.lr_find(show_plot=True, max_epochs=1)

In [13]:
learner1.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
4800/4800 [==============================] - 92902s 19s/step - loss: 1.5234 - accuracy: 0.3859 - val_loss: 1.1424 - val_accuracy: 0.5874
Epoch 2/3
4800/4800 [==============================] - 94159s 20s/step - loss: 0.9496 - accuracy: 0.6642 - val_loss: 0.7233 - val_accuracy: 0.7554
Epoch 3/3
4800/4800 [==============================] - 92717s 19s/step - loss: 0.4948 - accuracy: 0.8353 - val_loss: 0.5459 - val_accuracy: 0.8288


In [14]:
learner1.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

     disgust       0.77      0.82      0.80      1185
       angry       0.87      0.83      0.85      1192
       happy       0.90      0.89      0.90      1171
    surprise       0.88      0.86      0.87      1223
        fear       0.71      0.67      0.69      1236
         sad       0.85      0.90      0.87      1193

    accuracy                           0.83      7200
   macro avg       0.83      0.83      0.83      7200
weighted avg       0.83      0.83      0.83      7200



array([[ 975,   38,   23,   25,  103,   21],
       [  77,  994,    8,   14,   72,   27],
       [  28,   11, 1048,   17,   40,   27],
       [  33,   17,   12, 1052,   70,   39],
       [ 121,   69,   70,   73,  823,   80],
       [  29,   17,    9,   13,   50, 1075]], dtype=int64)

In [ ]:
learner1.evaluate()


In [ ]:
model.save("m-Bert_model")